This will be my GAN model used to create semi-realistic face images using the celeb A dataset.  First I will import all of the relevant libraries 

In [1]:
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torchvision import datasets

Next I will create a seed for this torch model in order to have reproducibility for the model.  We will also use deterministic algorithms to true as well to again ensure reproducibility of the results. 

In [2]:
manualSeed = 999 # Random choice for reproducing same results
print("Random Seed: ", manualSeed) # Print out the seed 
random.seed(manualSeed) # Set the seed 
torch.manual_seed(manualSeed) # Set the seed in pytorch
torch.use_deterministic_algorithms(True) # Turn on the deterministic algorithms 

Random Seed:  999


Next we will setup the hyperparameters for the model.  These include the dataset directory path, the number of workers for the dataloader, the batch size for training, the size of the images.  We also set the number of input channels for the model (3 for colour images), the latent vector size for the generator input, the feature map sizes for the generator and the discriminator.  The number of epochs or training cycles, the learning rate of the model, the Beta1 paramentere for the Adam optimiser and the number of available GPU's 

In [6]:
dataroot = './CelebA' # Root directory for dataset
workers = 2 # Number of workers for dataloader
batch_size = 128 # Batch size during training
image_size = 64 # Spatial size of training images. All images will be resized to this. #   size using a transformer.
nc = 3 # Number of channels in the training images. For color images this is 3
nz = 100 # Size of z latent vector (i.e. size of generator input)
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator
num_epochs = 5 # Number of training epochs, arbitrary choice mainly for time 
lr = 0.0002 # Learning rate for optimizers, we want to avoid overshooting 
beta1 = 0.5 # Beta1 hyperparameter for Adam optimizers
ngpu = 1 # Number of GPUs available. Use 0 for CPU mode.

Next we will want to load in our data for the celebA dataset.  I will have it downloaded locally in my the same directoty as my model. in the CelebA directory it has the following structure. 

./CelebA
    -> ./img_align_celeba
    -> identity_CelebA.txt
    -> list_attr_celeba.txt
    -> list_bbox_celeba.txt
    -> list_eval_partition.txt
    -> list_landmarks_align.txt
    -> list_landmarks_celeba.txt

Then we load this data into an ImageFolder dset, apply our transforms and then load the new dataset into the dataloader we will use for the model 

In [7]:
# Use an image folder to store the local data and load it in 
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# The transform to apply to the data for better overall results
transform=transforms.Compose([transforms.Resize(image_size),
                              transforms.CenterCrop(image_size),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ])

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './CelebA'

Now we will setup the gpu, as the device to use for the model, using the parrelism of the GPU will be vital for this model due to how much processing that we need to do. 

In [ ]:
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")# Decide which device we want to run on

Now lets print out some pretty images to look at 

In [ ]:
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images (So Pretty!)")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

Next we wanna set the weights for the models to use that is both the netG and netD (Generator and Discriminator Neural Networks).  

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)